In [1]:
# importing libraries
import pandas as pd
from surprise import Dataset
from surprise import Reader

In [2]:
# read the ratings into a pandas dataframe and then into a surprise dataset
df = pd.read_csv('ml-latest-small/ml-latest-small/ratings.csv')
df.drop(['timestamp'], axis=1, inplace=True)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[["userId", "movieId", "rating"]], reader)

In [ ]:
# split the data into 5 folds for cross-validation
